# 1. Khai báo thư viện

In [ ]:
import os
from collections import OrderedDict
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets
import pytorch_lightning as pl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.datasets import MNIST,FashionMNIST
PATH_DATASETS = 'data'
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count() / 2)

# 2. Định nghĩa module nạp dữ liệu


In [ ]:
class FMNISTDataModule(LightningDataModule):

    def __init__(
            self,
            data_dir: str = PATH_DATASETS,
            batch_size: int = BATCH_SIZE,
            num_workers: int = NUM_WORKERS,
    ):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transform = transforms.Compose({
            transforms.Grayscale(),
            # convert image or numpy.array (HxWxC) in the range [0,225] to in the range[0,1]
            transforms.ToTensor(),
            # Normalize a tensor image with mean and standard deviation
            transforms.Normalize((0.1307,), (0.3081,)),
        })

    def prepare_data(self):
        # download
        FashionMNIST(self.data_dir, train=True, download=True)
        FashionMNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            feminist_full = FashionMNIST(
                self.data_dir, train=True,
                transform=self.transform)
            self.femnist_train, self.femnist_val = random_split(feminist_full, [55000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.fmnist_test = FashionMNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(
                self.femnist_train,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(self.femnist_val, batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.fmnist_test, batch_size=self.batch_size, num_workers=self.num_workers)
